In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Administrator\\Documents\\GitHub\\Text-Summarization-NLP-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Administrator\\Documents\\GitHub\\Text-Summarization-NLP-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
import os
from textSummarizer import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\ProgramData\anaconda3\envs\summary\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-22 11:17:55,758: INFO: config: PyTorch version 2.3.0 available.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], truncation=True, padding=True,  max_length=512, return_tensors='pt')
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], truncation=True, padding=True, max_length=128, return_tensors='pt')

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transfromation = DataTransformation(config=data_transformation_config)
    data_transfromation.convert()
except Exception as e:
    raise e

[2024-05-22 11:17:55,941: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-22 11:17:55,944: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-22 11:17:55,945: INFO: common: created directory at: artifacts]
[2024-05-22 11:17:55,946: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]


ValueError: max_lenght is not a valid PaddingStrategy, please select one of ['longest', 'max_length', 'do_not_pad']